In [1]:
import numpy as np
import trimesh
import plotly.graph_objects as go
import imageio.v2 as imageio
from io import BytesIO

In [2]:
FPS = 30
DURATION = 5
NFRAMES = FPS * DURATION

IMG_W, IMG_H = 800, 600

# Function Libs

In [3]:
def lis_mesh_read(mesh_lis):
    meshes = []
    for i in range(len(mesh_lis)):
        mesh = trimesh.load(mesh_lis[i])
        meshes.append(mesh)
    return meshes

In [4]:
def xy_radius_limit_from_vertices(V, pad_ratio=0.05):
    r = np.sqrt(V[:,0]**2 + V[:,1]**2)
    rmax = float(r.max()) * (1.0 + pad_ratio)
    zmin = float(V[:,2].min())
    zmax = float(V[:,2].max())
    zpad = (zmax - zmin) * pad_ratio
    return rmax, zmin - zpad, zmax + zpad

In [5]:
def global_xyz_range(mesh_list, pad_ratio=0.05):
    """
    Return:
      xyz_min: (3,)
      xyz_max: (3,)
    pad_ratio: padding ratio relative to the span of the bounding box
    """
    rmaxs = []
    mins = []
    maxs = []
    for m in mesh_list:
        # bounds: [[xmin,ymin,zmin],[xmax,ymax,zmax]]
        V = np.asarray(m.vertices, dtype=float)
        rmax, zmin, zmax = xy_radius_limit_from_vertices(V, pad_ratio=pad_ratio)
        rmaxs.append(rmax)
        mins.append(zmin)
        maxs.append(zmax)
  
    rmax_global = max(rmaxs)
    zmin_global = min(mins)
    zmax_global = max(maxs)

    return rmax_global, zmax_global, zmin_global

In [ ]:
def generate_rotating_gif(mesh, gif_path, paras, mesh_color):
    V0 = np.asarray(mesh.vertices)
    F = np.asarray(mesh.faces) if hasattr(mesh, 'faces') and mesh.faces is not None and len(mesh.faces) > 0 else None
    rmax = paras[0]
    zmax = paras[1]
    zmin = paras[2]

    # -----------------------
    # Fixed camera
    # -----------------------
    fixed_camera = dict(
        eye=dict(x=1.8, y=1.8, z=1.2),
        center=dict(x=0.0, y=0.0, z=0.0),
        up=dict(x=0.0, y=0.0, z=1.0)
    )

    # -----------------------
    # Initial Figure
    # -----------------------
    traces = []

    if F is not None and len(F) > 0:
        # If have faces：Render mesh
        traces.append(
            go.Mesh3d(
                x=V0[:, 0], y=V0[:, 1], z=V0[:, 2],
                i=F[:, 0], j=F[:, 1], k=F[:, 2],
                color=mesh_color,
                opacity=1.0
            )
        )
    else:
        # If no faces: Render point cloud
        traces.append(
            go.Scatter3d(
                x=V0[:, 0], y=V0[:, 1], z=V0[:, 2],
                mode='markers',
                marker=dict(size=2, color=mesh_color)
            )
        )

    fig = go.Figure(data=traces)

    fig.update_layout(scene=dict(aspectmode="cube", 
                                camera=fixed_camera,
                                xaxis=dict(range=[-rmax, rmax]),
                                yaxis=dict(range=[-rmax, rmax]),
                                zaxis=dict(range=[zmin, zmax]),),
                                margin=dict(l=0, r=0, t=0, b=0),
                                showlegend=False)

    # -----------------------
    # Write GIF
    # -----------------------
    duration_per_frame = 1.0 / FPS

    with imageio.get_writer(gif_path, mode="I", duration=duration_per_frame, loop=0) as writer:
        for t in range(NFRAMES):
            theta = 2 * np.pi * (t / NFRAMES)
            c, s = np.cos(theta), np.sin(theta)

            Rz = np.array([[c, -s, 0.0],
                        [s,  c, 0.0],
                        [0.0, 0.0, 1.0]], dtype=float)

            V = V0 @ Rz.T

            fig.data[0].x = V[:, 0]
            fig.data[0].y = V[:, 1]
            fig.data[0].z = V[:, 2]
            fig.data[0].color = mesh_color
            
            png_bytes = fig.to_image(format="png", width=IMG_W, height=IMG_H, scale=1)
            frame = imageio.imread(BytesIO(png_bytes))
            writer.append_data(frame)

    print("GIF saved to:", gif_path)

# Gif Generation

In [7]:
object_lis = ["chips", "cylinder", "cube","asym"]
CAD_color = 'red'
raw_color = 'lightgray'
aligned_color = 'lightblue'
mesh_lis = []
gif_path = []
color_lis = []
limit_lis = []
for i in object_lis:
    object_name = i
    raw_mesh = f"meshes/{object_name}_object/generated_meshes/RAW_{object_name}.STL"
    aligned_mesh = f"meshes/{object_name}_object/generated_meshes/Aligned_{object_name}.STL"

    raw_gif = f"meshes/{object_name}_object/gifs/RAW_{object_name}.gif"
    aligned_gif = f"meshes/{object_name}_object/gifs/Aligned_{object_name}.gif"
    if object_name == "chips":
        object_meshes_lis = [raw_mesh, aligned_mesh]
        gif_path.extend([raw_gif, aligned_gif])
        color_lis.extend([raw_color, aligned_color])
    else:
        cad_mesh = f"meshes/{object_name}_object/generated_meshes/{object_name}_original.STL"
        cad_gif = f"meshes/{object_name}_object/gifs/CAD_{object_name}.gif"
        object_meshes_lis = [raw_mesh, aligned_mesh, cad_mesh]
        
        gif_path.extend([raw_gif, aligned_gif, cad_gif])
        color_lis.extend([raw_color, aligned_color, CAD_color])
    mesh_lis.extend(object_meshes_lis)
    object_meshes = lis_mesh_read(mesh_lis)
    rmax, zmax, zmin = global_xyz_range(object_meshes, pad_ratio=0.1)
    limit_lis.extend([(rmax, zmax, zmin)] * len(object_meshes_lis))

In [9]:
meshes = lis_mesh_read(mesh_lis)
rmax, zmax, zmin = global_xyz_range(meshes, pad_ratio=0.1)
for i in range(len(mesh_lis)):
    generate_rotating_gif(meshes[i], gif_path[i], limit_lis[i] ,color_lis[i])

GIF saved to: meshes/chips_object/gifs/RAW_chips.gif
GIF saved to: meshes/chips_object/gifs/Aligned_chips.gif
GIF saved to: meshes/cylinder_object/gifs/RAW_cylinder.gif
GIF saved to: meshes/cylinder_object/gifs/Aligned_cylinder.gif
GIF saved to: meshes/cylinder_object/gifs/CAD_cylinder.gif
GIF saved to: meshes/cube_object/gifs/RAW_cube.gif
GIF saved to: meshes/cube_object/gifs/Aligned_cube.gif
GIF saved to: meshes/cube_object/gifs/CAD_cube.gif
GIF saved to: meshes/asym_object/gifs/RAW_asym.gif
GIF saved to: meshes/asym_object/gifs/Aligned_asym.gif
GIF saved to: meshes/asym_object/gifs/CAD_asym.gif
